In [1]:
# !pip install open3d

In [3]:
ply_file_path = './i8/longdress/longdress/Ply/'
ply_file_name = 'longdress_vox10_1311.ply'

In [1]:
import numpy as np
import open3d as o3d

def main():
    pts = np.random.randint(0, 100, (100, 3))

    # whether to write in binary or text format
    write_text = True

    # use open3d
    use_o3d(pts, write_text)


def use_o3d(pts, write_text):
    pcd = o3d.geometry.PointCloud()

    # the method Vector3dVector() will convert numpy array of shape (n, 3) to Open3D format.
    # see http://www.open3d.org/docs/release/python_api/open3d.utility.Vector3dVector.html#open3d.utility.Vector3dVector
    pcd.points = o3d.utility.Vector3dVector(pts)

    # http://www.open3d.org/docs/release/python_api/open3d.io.write_point_cloud.html#open3d.io.write_point_cloud
    my_pts = ply_file_path+ply_file_name
    o3d.io.write_point_cloud(my_pts, pcd, write_ascii=write_text)

    # read ply file
    pcd = o3d.io.read_point_cloud(my_pts)

    # visualize
    # o3d.visualization.draw_geometries([pcd])

In [6]:
main()

[Open3D WARNING] Write PLY failed: unable to open file: Users/chenli/research/point cloud/i8/longdress/longdress/Ply/longdress_vox10_1311.ply
[Open3D WARNING] Read PLY failed: unable to open file: Users/chenli/research/point cloud/i8/longdress/longdress/Ply/longdress_vox10_1311.ply


RPly: Unable to create file
RPly: Unable to open file


# read ply

In [5]:
my_pts = ply_file_path+ply_file_name
pcd = o3d.io.read_point_cloud(my_pts)

NameError: name 'o3d' is not defined

In [34]:
pcd

PointCloud with 854949 points.

In [18]:
# visualize
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [13]:
# Pass numpy array to Open3D.o3d.geometry.PointCloud and visualize
xyz = np.random.rand(100, 3)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.io.write_point_cloud("./data.ply", pcd)

True

In [16]:
# o3d.visualization.draw_geometries([pcd])
o3d.visualization.draw_geometries_with_editing([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# vivo

In [99]:
%run tgn_utils.py

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'l') according to the casting rule ''same_kind''

In [101]:
def occlusion_mapping(R_c, alpha, beta, S_c):
    """
    Maps R(c) to an integer value based on given thresholds.

    :param R_c:   The R(c) value to map
    :param alpha: List or tuple of alpha values [α0, α1, α2]
    :param beta:  The beta value
    :param S_c:   The S(c) value
    :return:      The mapped integer value
    """
    thresholds = [a * (beta ** (S_c - 1)) for a in alpha]
    print(thresholds)

    if R_c < thresholds[0]:
        return 0
    elif thresholds[0] <= R_c < thresholds[1]:
        return 1
    elif thresholds[1] <= R_c < thresholds[2]:
        return 2
    elif thresholds[2] <= R_c:
        return 3

# Example usage:
# alpha_values = [alpha0, alpha1, alpha2]  # Replace with actual values for α0, α1, and α2
# beta_value = beta  # Replace with the actual value for β
# S_c_value = S_c  # Replace with the actual value for S(c)
# R_c_value = R_c  # Replace with the actual value for R(c)

# result = occlusion_mapping(R_c_value, alpha_values, beta_value, S_c_value)
# print(result)


In [102]:
alpha0 = 0.6
alpha1=1
alpha2=3
beta = 0.8
S_c =1
R_c = 1


alpha_values = [alpha0, alpha1, alpha2]  # Replace with actual values for α0, α1, and α2
beta_value = beta  # Replace with the actual value for β
S_c_value = S_c  # Replace with the actual value for S(c)
R_c_value = R_c  # Replace with the actual value for R(c)

result = occlusion_mapping(R_c_value, alpha_values, beta_value, S_c_value)
print(result)


[0.6, 1.0, 3.0]
2



# To implement a Ray-Box Intersection algorithm in Python, which counts the number of surrounding cells (out of the 26 neighbors in a 3x3x3 grid excluding the center cell) that meet the occlusion criteria, we can follow these steps:

Define the ray starting from the viewpoint and going through the center of the cell c.
Check the intersection of this ray with each of the 26 neighboring cells.
Count those cells that intersect with the ray and are closer to the viewpoint than cell c. -->

In [103]:
def ray_box_intersection(ray_origin, ray_direction, box_min, box_max):
    """
    Check if a ray intersects with a box (AABB).
    :param ray_origin: Origin of the ray (np.array)
    :param ray_direction: Direction of the ray (np.array)
    :param box_min: Minimum vertex of the box (np.array)
    :param box_max: Maximum vertex of the box (np.array)
    :return: Boolean indicating if there is an intersection
    """
    t_min = (box_min - ray_origin) / ray_direction
    t_max = (box_max - ray_origin) / ray_direction
    t1 = np.minimum(t_min, t_max)
    t2 = np.maximum(t_min, t_max)
    t_near = np.max(t1)
    t_far = np.min(t2)

    if t_near > t_far or t_far < 0:
        return False

    return True

def occlusion_count(viewpoint, target_cell_center, cell_size, target_distance):
    """
    Count the number of surrounding cells that meet the occlusion criteria.
    :param viewpoint: Viewpoint coordinates and orientation (X, Y, Z, yaw, pitch, roll)
    :param target_cell_center: Center of the target cell 'c'
    :param cell_size: Size of each cell
    :param target_distance: Distance from the viewpoint to the target cell 'c'
    :return: Number of cells occluding the target cell 'c'
    """
    count = 0
    ray_origin = np.array(viewpoint[:3])
    ray_direction = target_cell_center - ray_origin  # Assumes already normalized if needed
    # ray_direction /= np.linalg.norm(ray_direction)   # Normalize the ray direction
    # Define the bounds of the surrounding cells
    offsets = [-cell_size, 0, cell_size]
    for dx in offsets:
        for dy in offsets:
            for dz in offsets:
                if dx == dy == dz == 0:
                    continue  # Skip the center cell itself
                
                neighbor_center = target_cell_center + np.array([dx, dy, dz])
                box_min = neighbor_center - np.array([cell_size/2] * 3)
                box_max = neighbor_center + np.array([cell_size/2] * 3)
                distance_to_neighbor = np.linalg.norm(neighbor_center - ray_origin)
                # import pdb;pdb.set_trace()
                if ray_box_intersection(ray_origin, ray_direction, box_min, box_max) and distance_to_neighbor < target_distance:
                    count += 1

    return count



In [110]:
import numpy as np
# Example usage:
viewpoint = (1, 1, 1)  # Example viewpoint with XYZ and yaw, pitch, roll
target_cell_center = np.array([5, 5, 0])  # Center of the target cell
cell_size = 1.0  # Size of each cell
target_distance = np.linalg.norm(target_cell_center - np.array(viewpoint[:3]))

occluding_cells = occlusion_count(viewpoint, target_cell_center, cell_size, target_distance)
print(f"Number of occluding cells: {occluding_cells}")


Number of occluding cells: 3


In [36]:
# target_distance

# view frustum

In [47]:
import numpy as np

def ypr_to_direction(yaw, pitch, roll):
    # Assuming the angles are given in degrees and the roll is not used for the direction
    yaw_rad = np.radians(yaw)
    pitch_rad = np.radians(pitch)
    
    # Calculate the direction vector
    x = np.cos(pitch_rad) * np.cos(yaw_rad)
    y = np.sin(pitch_rad)
    z = np.cos(pitch_rad) * np.sin(yaw_rad)
    
    return np.array([x, y, z])


In [48]:
def get_view_frustum_cells(viewpoint, yaw, pitch, roll, fov_y, aspect_ratio, near_clip, far_clip, cell_size, grid_size):
    direction = ypr_to_direction(yaw, pitch, roll)
    
    # Calculate the tangent of the half FoV
    tan_fov_y = np.tan(np.radians(fov_y / 2))
    tan_fov_x = aspect_ratio * tan_fov_y
    
    # Define the frustum bounds relative to the viewpoint
    half_height_near = tan_fov_y * near_clip
    half_width_near = tan_fov_x * near_clip
    half_height_far = tan_fov_y * far_clip
    half_width_far = tan_fov_x * far_clip
    
    target_cells = []
    
    # Loop through each cell in the grid
    for x in range(grid_size[0]):
        for y in range(grid_size[1]):
            for z in range(grid_size[2]):
                # Calculate the center of the cell
                cell_center = np.array([x, y, z]) * cell_size + cell_size / 2
                
                # Vector from the viewpoint to the cell center
                vec = cell_center - viewpoint
                
                # Distance from the viewpoint to the cell center
                dist = np.linalg.norm(vec)
                
                # Normalize the vector
                norm_vec = vec / dist
                
                # Angle between the view direction and the vector
                angle_x = np.arctan2(norm_vec[0], norm_vec[2])
                angle_y = np.arctan2(norm_vec[1], norm_vec[2])
                
                # Check if within frustum bounds
                if (-half_width_near <= dist * np.tan(angle_x) <= half_width_near and
                    -half_height_near <= dist * np.tan(angle_y) <= half_height_near and
                    near_clip <= dist <= far_clip):
                    target_cells.append((x, y, z))
    
    return target_cells


In [49]:
X,Y,Z,YAW,PITCH,ROLL = 0.05,1.7868,-1.0947,6.9163,350.8206,359.9912

viewpoint = np.array([X,Y,Z])  # Replace with your viewpoint coordinates
yaw = YAW  # Replace with your yaw angle
pitch = PITCH  # Replace with your pitch angle
roll = ROLL  # Replace with your roll angle (usually not used for frustum calculation)
fov_y = 45.0  # Field of view angle in degrees
aspect_ratio = 1 / 1  # Replace with your display's aspect ratio
near_clip = 1  # Replace with your near clipping plane distance
far_clip = 100  # Replace with your far clipping plane distance
cell_size = 1  # Replace with the size of each cell
grid_size = (1, 1, 1)  # Replace with the size of your grid

target_cells = get_view_frustum_cells(viewpoint, yaw, pitch, roll, fov_y, aspect_ratio, near_clip, far_clip, cell_size, grid_size)


In [50]:
target_cells

[]

In [52]:
import numpy as np

def get_view_frustum_cells(viewpoint, orientation, fov_y, aspect_ratio, near_clip, far_clip, cell_size):
    """
    Get all target cells within the view frustum.

    :param viewpoint: (X, Y, Z) coordinates of the viewpoint.
    :param orientation: (yaw, pitch, roll) orientation of the viewpoint.
    :param fov_y: Field of View angle in Y dimension in degrees.
    :param aspect_ratio: Aspect ratio (width/height) of the view frustum.
    :param near_clip: Distance from the viewer to the near clipping plane.
    :param far_clip: Distance from the viewer to the far clipping plane.
    :param cell_size: Size of each cell in the grid.
    :return: List of cell centers within the view frustum.
    """
    # Convert orientation to radians
    yaw, pitch, roll = np.radians(orientation)

    # Calculate the view direction vector
    direction = np.array([
        np.cos(pitch) * np.sin(yaw),
        np.sin(pitch),
        np.cos(pitch) * np.cos(yaw)
    ])

    # Calculate half-angles for frustum planes
    half_fov_y = np.radians(fov_y / 2)
    half_fov_x = np.arctan(np.tan(half_fov_y) * aspect_ratio)

    # Calculate frustum plane normals (assuming Y up, X right, Z into the screen)
    up = np.array([0, 1, 0])
    right = np.cross(up, direction)
    up = np.cross(direction, right)
    
    normals = {
        'near': direction,
        'far': -direction,
        'top': np.cross(right, np.array([np.sin(half_fov_y), np.cos(half_fov_y), 0])),
        'bottom': np.cross(np.array([-np.sin(half_fov_y), np.cos(half_fov_y), 0]), right),
        'left': np.cross(up, np.array([np.cos(half_fov_x), 0, -np.sin(half_fov_x)])),
        'right': np.cross(np.array([np.cos(half_fov_x), 0, np.sin(half_fov_x)]), up)
    }

    # Define frustum bounds
    frustum_bounds = {
        'near': viewpoint + direction * near_clip,
        'far': viewpoint + direction * far_clip
    }

    # Function to check if a point is inside the frustum
    def is_inside_frustum(point):
        for key, normal in normals.items():
            d = np.dot(normal, point - frustum_bounds.get(key, viewpoint))
            if (key == 'near' and d < 0) or (key == 'far' and d > 0):
                return False
            elif d < 0:
                return False
        return True

    # Iterate through cells and check if they are in the frustum
    cells_in_frustum = []
    for x in range(int(viewpoint[0] - far_clip), int(viewpoint[0] + far_clip), cell_size):
        for y in range(int(viewpoint[1] - far_clip), int(viewpoint[1] + far_clip), cell_size):
            for z in range(int(viewpoint[2] - far_clip), int(viewpoint[2] + far_clip), cell_size):
                cell_center = np.array([x + cell_size / 2, y + cell_size / 2, z + cell_size / 2])
                if is_inside_frustum(cell_center):
                    cells_in_frustum.append(cell_center)
    return cells_in_frustum

# Example usage:
viewpoint = np.array([0, 0, 0])  # Viewpoint coordinates
orientation = (0, 0, 0)  # View direction (yaw, pitch, roll)
fov_y = 45  # Field of View in Y dimension
aspect_ratio = 16/9  # Aspect ratio
near_clip = 1  # Near clipping distance
far_clip = 100  # Far clipping distance
cell_size = 1  # Size of each cell

cells = get_view_frustum_cells(viewpoint, orientation, fov_y, aspect_ratio, near_clip, far_clip, cell_size)
print(f"Number of cells")


Number of cells


# FoV tiles version 2

In [96]:
import numpy as np
from math import sin, cos, radians, tan

def get_view_frustum_cells(viewpoint, orientation, fov_y, aspect_ratio, grid_size, occupied_cells):
    yaw, pitch, roll = orientation
    # Convert angles to radians
    yaw, pitch, roll = radians(yaw), radians(pitch), radians(roll)
    
    # Compute the rotation matrix from the orientation
    Rz_yaw = np.array([
        [cos(yaw), -sin(yaw), 0],
        [sin(yaw),  cos(yaw), 0],
        [0,             0,    1]
    ])

    Ry_pitch = np.array([
        [cos(pitch), 0, sin(pitch)],
        [0,          1,          0],
        [-sin(pitch),0, cos(pitch)]
    ])

    Rx_roll = np.array([
        [1, 0,           0],
        [0, cos(roll), -sin(roll)],
        [0, sin(roll),  cos(roll)]
    ])

    rotation_matrix = Rz_yaw @ Ry_pitch @ Rx_roll
    
    # Calculate half-angles for frustum bounds
    half_fov_y = radians(fov_y) / 2
    half_fov_x = half_fov_y * aspect_ratio
    
    # Define a lambda function to check if a point is in the frustum
    def in_frustum(point, view_matrix):
        # Transform the point into the view space
        transformed_point = view_matrix @ np.array([point[0], point[1], point[2], 1]).T
        
        # Check against the frustum planes
        x, y, z, w = transformed_point
        x /= w
        y /= w
        z /= w
        in_x = -tan(half_fov_x) * z <= x <= tan(half_fov_x) * z
        in_y = -tan(half_fov_y) * z <= y <= tan(half_fov_y) * z
        in_z = 0 < z  # Assuming the camera only looks forward
        
        return in_x and in_y and in_z
    
    # Create the view matrix from the rotation matrix and the translation vector
    view_matrix = np.identity(4)
    view_matrix[:3, :3] = rotation_matrix
    view_matrix[:3, 3] = -rotation_matrix @ viewpoint
    
    # Check each cell
    visible_cells = []
    for cell in occupied_cells:
        # Check if the occupied cell is in the frustum
        if in_frustum(cell, view_matrix):
            visible_cells.append(cell)
            
    return visible_cells

# Assuming a function `is_occupied(x, y, z)` that returns True if the cell is occupied
# def is_occupied(x, y, z):
#     # Placeholder function for determining if a cell is occupied
#     # Replace with your actual logic
#     return np.random.rand() > 0.5  # Randomly occupying cells for this example


# Define the grid size and the occupied cells
grid_size = 16
occupied_cells = [(x, y, z) for x in range(grid_size) for y in range(grid_size) for z in range(grid_size) if is_occupied(x, y, z)]

# Example usage of the function
viewpoint = (0, 0, 0)  # Center of the grid
orientation = (0, 0, 0)  # Looking in the positive Z direction
fov_y = 45  # Field of view in the Y dimension in degrees
aspect_ratio = 16/9  # Aspect ratio of the display


visible_cells = get_view_frustum_cells(viewpoint, orientation, fov_y, aspect_ratio, grid_size, occupied_cells)
print(f"Visible cells: {visible_cells}")


Visible cells: [(6, 0, 8), (6, 0, 9), (6, 1, 8), (6, 1, 9), (7, 0, 9), (7, 1, 9), (7, 2, 9), (7, 3, 9), (7, 4, 10)]


In [97]:
# len(visible_cells)
len(occupied_cells)

192

In [58]:
!pwd

/Users/chenli/research/point cloud


# tile occupied or not

In [91]:
import os

# Assuming 'base_path' is the directory containing the dataset
base_path = '/Users/chenli/research/point cloud/tile_data/recons/1051/'

def is_occupied(x, y, z, resolutions=[1, 2, 3, 4, 5, 6]):
    """
    Check if a cell is occupied based on the existence of a corresponding file.
    
    :param frame_index: Frame index of the dataset
    :param x: X index of the cell
    :param y: Y index of the cell
    :param z: Z index of the cell
    :param resolutions: List of possible resolutions
    :return: True if the cell is occupied, False otherwise
    """
    frame_index=1051
    for resolution in resolutions:
        # Construct the file name based on the given parameters
        file_name = f'longdress_{frame_index:04d}_{x:02d}_{y:02d}_{z:02d}_{resolution}.ply'
        file_path = os.path.join(base_path, file_name)
        # print(file_path)
        # Check if the file exists
        if os.path.isfile(file_path):
            return True
    
    return False

# Example usage:
frame_index = 1051
x, y, z = 5, 7, 8
occupied = is_occupied(x, y, z)
print(f"The cell at ({x}, {y}, {z}) is {'occupied' if occupied else 'not occupied'}.")

The cell at (5, 7, 8) is occupied.


In [76]:
import numpy as np
import os

# Assuming 'base_path' is the directory containing the dataset
# base_path = '/path/to/your/dataset/'
frame_index = 1051  # The frame index you're interested in
resolutions = [1, 2, 3, 4, 5, 6]  # The list of possible resolutions

# Initialize a 3D NumPy array with zeros (not occupied)
occupied_array = np.zeros((16, 16, 16), dtype=bool)

# Populate the array with occupancy information
for x in range(16):
    for y in range(16):
        for z in range(16):
            for resolution in resolutions:
                file_name = f'longdress_{frame_index:04d}_{x:02d}_{y:02d}_{z:02d}_{resolution}.ply'
                file_path = os.path.join(base_path, file_name)
                # print(file_path)
                if os.path.isfile(file_path):                    
                    occupied_array[x, y, z] = True
                    break  # No need to check other resolutions if the cell is occupied

# Now, occupied_array contains True for occupied cells and False for empty cells
# Example usage to check if a cell is occupied:
x, y, z = 5, 7, 6
is_occupied = occupied_array[x, y, z]
print(f"Cell ({x}, {y}, {z}) is {'occupied' if is_occupied else 'not occupied'}.")



Cell (5, 7, 6) is occupied.


In [77]:
occupied_array
# If you want to save the array to a file to use later without recalculating
np.save('occupied_array.npy', occupied_array)

# To load this array later, simply use:
# occupied_array = np.load('occupied_array.npy')


In [79]:
occupied_array.sum()

192